**Import libraries**

In [36]:
import transformers
from datasets import load_dataset
import pandas as pd
import numpy as np

**Load SQUAD Dataset**

In [5]:
datasets = load_dataset("squad")

Downloading:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/8.12M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset squad downloaded and prepared to C:\Users\algin\.cache\huggingface\datasets\squad\plain_text\1.0.0\d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [18]:
train_dataset = datasets["train"]
valid_dataset = datasets["validation"]

In [51]:
train_data = train_dataset.to_pandas()
valid_data = valid_dataset.to_pandas()

In [28]:
train_data.head()

,id,title,context,question,answers
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,"{'text': ['Saint Bernadette Soubirous'], 'answ..."
1,5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,"{'text': ['a copper statue of Christ'], 'answe..."
2,5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,"{'text': ['the Main Building'], 'answer_start'..."
3,5733be284776f41900661181,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,{'text': ['a Marian place of prayer and reflec...
4,5733be284776f4190066117e,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,{'text': ['a golden statue of the Virgin Mary'...


In [45]:
train_data.loc[0,"answers"]

{'text': array(['Saint Bernadette Soubirous'], dtype=object),
 'answer_start': array([515])}

**Split dict inside answers column to two separate columns and pop out the value from the list**

In [59]:
def split_answers_column(data):
    answers_column_split = data["answers"].apply(pd.Series)
    data = pd.concat([data,answers_column_split],axis=1)
    data.drop("answers",axis=1,inplace=True)
    data.rename({"text":"answer_text"},axis=1,inplace=True)
    data['answer_text'] = data['answer_text'].map(lambda x : x[0])
    data['answer_start'] = data['answer_start'].map(lambda x : x[0])
    return data

In [60]:
train_data_with_answer_info = split_answers_column(train_data)
valid_data_with_answer_info = split_answers_column(valid_data)

In [61]:
train_data_with_answer_info.head()

,id,title,context,question,answer_text,answer_start
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,Saint Bernadette Soubirous,515
1,5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,a copper statue of Christ,188
2,5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,the Main Building,279
3,5733be284776f41900661181,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,a Marian place of prayer and reflection,381
4,5733be284776f4190066117e,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,a golden statue of the Virgin Mary,92


In [62]:
train_data_with_answer_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87599 entries, 0 to 87598
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            87599 non-null  object
 1   title         87599 non-null  object
 2   context       87599 non-null  object
 3   question      87599 non-null  object
 4   answer_text   87599 non-null  object
 5   answer_start  87599 non-null  int64 
dtypes: int64(1), object(5)
memory usage: 4.0+ MB
